## Configuring cuDNN on Colab for YOLOv4

In [1]:
!/usr/local/cuda/bin/nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2019 NVIDIA Corporation
Built on Sun_Jul_28_19:07:16_PDT_2019
Cuda compilation tools, release 10.1, V10.1.243


In [2]:
!nvidia-smi

Thu Feb 25 18:01:29 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.39       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   53C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
%env compute_capability=60

env: compute_capability=60


## Downloading the code from the Dropbox

In [4]:
%cd /content/

/content


In [5]:
!wget https://www.dropbox.com/sh/xjssg151ppl0v2h/AACAam-nc2x0_X9eD7yL7XTba?dl=0

--2021-02-25 18:01:29--  https://www.dropbox.com/sh/xjssg151ppl0v2h/AACAam-nc2x0_X9eD7yL7XTba?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.6.18, 2620:100:601c:18::a27d:612
Connecting to www.dropbox.com (www.dropbox.com)|162.125.6.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /sh/raw/xjssg151ppl0v2h/AACAam-nc2x0_X9eD7yL7XTba [following]
--2021-02-25 18:01:29--  https://www.dropbox.com/sh/raw/xjssg151ppl0v2h/AACAam-nc2x0_X9eD7yL7XTba
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uca9a9c939f2f1b764ccb4ee6d61.dl.dropboxusercontent.com/zip_download_get/AtDV1N4LDP5zlHdCk7i5s7b02In1i8b7FthsHNn3MNDOdTlgSgv8K6oB7HUVPgFZBCXQvUY_7DckP1HAcss1J3gF55NFGdRtlhXp6SdzzWVbbQ [following]
--2021-02-25 18:01:30--  https://uca9a9c939f2f1b764ccb4ee6d61.dl.dropboxusercontent.com/zip_download_get/AtDV1N4LDP5zlHdCk7i5s7b02In1i8b7FthsHNn3MNDOdTlgSgv8K6oB7HUVPgFZBCXQvUY_7DckP1HAc

In [6]:
!mkdir tensorflow-yolov4-tflite

In [7]:
!unzip /content/AACAam-nc2x0_X9eD7yL7XTba?dl=0 -d /content/tensorflow-yolov4-tflite

Archive:  /content/AACAam-nc2x0_X9eD7yL7XTba?dl=0
mapname:  conversion of  failed
   creating: /content/tensorflow-yolov4-tflite/mAP/
   creating: /content/tensorflow-yolov4-tflite/core/
   creating: /content/tensorflow-yolov4-tflite/data/
   creating: /content/tensorflow-yolov4-tflite/scripts/
 extracting: /content/tensorflow-yolov4-tflite/LICENSE  
   creating: /content/tensorflow-yolov4-tflite/android/
 extracting: /content/tensorflow-yolov4-tflite/cars.mp4  
 extracting: /content/tensorflow-yolov4-tflite/train.py  
 extracting: /content/tensorflow-yolov4-tflite/README.md  
   creating: /content/tensorflow-yolov4-tflite/mAP/extra/
 extracting: /content/tensorflow-yolov4-tflite/detect.py  
   creating: /content/tensorflow-yolov4-tflite/data/video/
   creating: /content/tensorflow-yolov4-tflite/detections/
 extracting: /content/tensorflow-yolov4-tflite/mAP/main.py  
   creating: /content/tensorflow-yolov4-tflite/scripts/voc/
   creating: /content/tensorflow-yolov4-tflite/data/images/


## Installing the requirements

In [8]:
%cd /content/tensorflow-yolov4-tflite
!pip install -r requirements-gpu.txt

/content/tensorflow-yolov4-tflite
     |████████████████████████████████| 320.4MB 53kB/s 
     |████████████████████████████████| 28.7MB 120kB/s 
     |████████████████████████████████| 20.1MB 96kB/s 
     |████████████████████████████████| 3.0MB 48.8MB/s 
     |████████████████████████████████| 460kB 49.0MB/s 
ERROR: tensorflow 2.4.1 has requirement numpy~=1.19.2, but you'll have numpy 1.18.5 which is incompatible.
ERROR: tensorflow 2.4.1 has requirement tensorboard~=2.4, but you'll have tensorboard 2.2.2 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: numpy 1.19.5
    Uninstalling numpy-1.19.5:
      Successfully uninstalled numpy-1.19.5
  Found existing installation: tensorboard 2.4.1
    Uninstalling tensorboard-2.4.1:
      Successfully uninstalled te

In [9]:
!sudo apt install tesseract-ocr
!pip install pytesseract

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 3 newly installed, 0 to remove and 13 not upgraded.
Need to get 4,795 kB of archives.
After this operation, 15.8 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 tesseract-ocr-eng all 4.00~git24-0e00fe6-1.2 [1,588 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 tesseract-ocr-osd all 4.00~git24-0e00fe6-1.2 [2,989 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic/universe amd64 tesseract-ocr amd64 4.00~git2288-10f4998a-2 [218 kB]
Fetched 4,795 kB in 1s (5,397 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl

In [10]:
import pytesseract
!which tesseract /usr/bin/tesseract
pytesseract.pytesseract.tesseract_cmd = ( r'/usr/bin/tesseract')

/usr/bin/tesseract
/usr/bin/tesseract


## Running only the detection step on the video

In [11]:
# Converting the model to be compatable tensorflow framework
# !python /content/tensorflow-yolov4-tflite/save_model.py --weights /content/tensorflow-yolov4-tflite/data/custom.weights --output ./checkpoints/custom-416 --input_size 416 --model yolov4

In [12]:
%cd /content/tensorflow-yolov4-tflite

/content/tensorflow-yolov4-tflite


In order to run the code on a different video you copy the path to your video instead of /content/tensorflow-yolov4-tflite/cars.mp4 in the cell below.

The result will be saved in the detections folder.

In [13]:
!python detect_video.py --weights ./checkpoints/custom-416 --size 416 --model yolov4 --video /content/tensorflow-yolov4-tflite/cars.mp4  --output ./detections/detect.avi 

FPS: 0.13
FPS: 28.60
FPS: 28.33
FPS: 27.73
FPS: 28.00
FPS: 29.75
FPS: 28.53
FPS: 29.95
FPS: 28.42
FPS: 27.00
FPS: 29.21
FPS: 27.75
FPS: 29.39
FPS: 28.20
FPS: 28.06
FPS: 28.91
FPS: 27.36
FPS: 29.83
FPS: 28.47
FPS: 28.74
FPS: 29.56
FPS: 29.54
FPS: 28.45
FPS: 29.03
FPS: 29.63
FPS: 29.53
FPS: 29.91
FPS: 28.96
FPS: 29.39
FPS: 28.16
FPS: 29.67
FPS: 29.32
FPS: 28.60
FPS: 28.04
FPS: 28.17
FPS: 28.24
FPS: 30.20
FPS: 28.87
FPS: 27.89
FPS: 29.45
FPS: 29.01
FPS: 29.32
FPS: 28.47
FPS: 29.57
FPS: 28.66
FPS: 30.02
FPS: 28.87
FPS: 28.72
FPS: 30.17
FPS: 27.56
FPS: 29.10
FPS: 26.07
FPS: 28.72
FPS: 28.55
FPS: 27.79
FPS: 27.01
FPS: 28.87
FPS: 28.01
FPS: 27.96
FPS: 26.89
FPS: 29.59
FPS: 29.86
FPS: 27.69
FPS: 27.98
FPS: 29.39
FPS: 29.10
FPS: 29.59
FPS: 27.54
FPS: 29.00
FPS: 29.96
FPS: 29.15
FPS: 29.10
FPS: 29.51
FPS: 29.57
FPS: 27.86
FPS: 28.89
FPS: 29.99
FPS: 28.92
FPS: 29.97
FPS: 28.39
FPS: 28.77
FPS: 29.97
FPS: 28.53
FPS: 28.15
FPS: 28.94
FPS: 29.38
FPS: 26.62
FPS: 28.98
FPS: 28.59
FPS: 28.37
FPS: 29.29


## Detection and recognition

In [14]:
!python detect_video.py --weights ./checkpoints/custom-416 --size 416 --model yolov4 --video /content/tensorflow-yolov4-tflite/cars.mp4  --output ./detections/detect_and_recon.avi --plate

License Plate #:  
FPS: 0.38
License Plate #:  
FPS: 27.96
License Plate #:  
FPS: 27.54
License Plate #:  
FPS: 28.46
License Plate #:  3
FPS: 2.91
License Plate #:  3
FPS: 5.38
License Plate #:  
FPS: 25.46
License Plate #:  
FPS: 28.42
License Plate #:  
FPS: 28.38
License Plate #:  
FPS: 28.39
License Plate #:  
FPS: 28.96
License Plate #:  
FPS: 28.54
License Plate #:  
FPS: 28.17
License Plate #:  
FPS: 28.20
License Plate #:  a
FPS: 5.08
License Plate #:  5
FPS: 5.42
License Plate #:  
FPS: 28.05
License Plate #:  a
FPS: 5.52
License Plate #:  
FPS: 27.29
FPS: 28.59
License Plate #:  2
FPS: 5.42
License Plate #:  
FPS: 27.92
License Plate #:  
FPS: 28.57
License Plate #:  
FPS: 28.00
License Plate #:  
FPS: 29.30
License Plate #:  
FPS: 5.50
License Plate #:  
FPS: 27.19
License Plate #:  
FPS: 25.92
License Plate #:  
FPS: 26.23
License Plate #:  
FPS: 26.42
License Plate #:  Z
FPS: 5.36
License Plate #:  
FPS: 27.40
License Plate #:  
FPS: 5.27
License Plate #:  
FPS: 25.16
Li